In [1]:
!pip install SpellChecker
!pip install swifter
!pip install pandarallel
!pip install pyspark
!pip install dask

     |████████████████████████████████| 3.9 MB 1.2 MB/s 
  Created wheel for SpellChecker: filename=spellchecker-0.4-py3-none-any.whl size=3966516 sha256=04174a4e8569ad0052816e5e90b53d987b4360ab6b4e1101df77b066c69ea19e
  Stored in directory: /root/.cache/pip/wheels/58/e9/48/b82b733a7a0a9cc52ed239ccf082ff33e2fbda71670ddd3349
  Created wheel for inexactsearch: filename=inexactsearch-1.0.2-py3-none-any.whl size=7112 sha256=8e626e90dcf93ccf3e8439b51c8891a0b8057a0f88f008e5c595814aa6176fdb
  Stored in directory: /root/.cache/pip/wheels/3c/f3/5d/77e8983e86af1c3c62b80d76de8020f560ce6367f1b2763b40
  Created wheel for silpa-common: filename=silpa_common-0.3-py3-none-any.whl size=8452 sha256=aa7f3a79416511fd08137135ec3f07084702a4e2208a7ec652940138da411a13
  Stored in directory: /root/.cache/pip/wheels/02/81/11/40ef1cf40f5c4021a214653ca3362914f0f9e14e8322f75f9b
  Created wheel for soundex: filename=soundex-1.1.3-py3-none-any.whl size=8861 sha256=37651de30333fb8405fef84b2905790acce255b6068a0ef4fdcc

In [2]:

import numpy as np 
import pandas as pd 
import nltk
from nltk.stem.isri import ISRIStemmer
import re
import time
import multiprocessing as mp
from multiprocessing import Pool
from multiprocessing import cpu_count
from dask import dataframe as dd
from dask.multiprocessing import get
import multiprocessing
import swifter
import threading

import dask.dataframe as dd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/whatsapp-group-chat/whatsapp_group_chat.txt


In [3]:
#######All the Function used to read the data, clean and pre-process it
st = ISRIStemmer() #Arabic stemmer
directory = "../input/whatsapp-group-chat/whatsapp_group_chat.txt"

#Reading Data
def read_data(file_directory):
    f = open(file_directory, "r")
    list_of_lists = []  
    for line in f:
      stripped_line = line.strip()
      line_list = stripped_line.split("-")
      list_of_lists.append(line_list)   
    list_of_lists.pop(0)
    f.close() 
    return list_of_lists

# turning it into a DataFrame
def pre(listoflists):
    df = pd.DataFrame(listoflists)
    df = df[[0, 1]]
    df = df.rename(columns={0: "Date", 1: "Chat"}, errors="raise")
    df.Chat = df.Chat.str.replace("[<@*&?].*[>@*&?]", "")
    df.Chat = df.Chat.str.replace(r'^.*?:', '') 
    df = df[df['Chat'] != ' '] #after the step above some cells ended having only void space
    #df.Chat = df.Chat.str.replace('5', 'kh')
    #df.Chat = df.Chat.str.replace('2', 'q')
    #df.Chat = df.Chat.str.replace('3', 'a') # I found that google translates 3 correctly to ع
    #df.Chat = df.Chat.str.replace('7', 'h')
    df.dropna(how = 'all', inplace=True) 
    return df

###Pre-Processing 101
#Removing Special Character
def remove_special(s):
    x = re.sub("[^0-9a-zA-Z]"r" ", s)
    x = re.sub(r'^\s*$', np.nan, s)
    return x

#processing 2-gram letters
# this function should be updated because it  takes only the first letter and change it to the assigned value
def duos(c):
    newstr = c
    sh = ("sh")
    kh = ("kh")
    for x in c:
        if x in sh:
            newstr = newstr.replace(x,"$")
        if x in kh:
            newstr = newstr.replace(x,"5")
    return newstr

#Removing Vowels
def anti_vowel(c):
    newstr = c
    vowels = ('ا', 'ي', 'و')    
    for x in c:
        if x in vowels:
            newstr = newstr.replace(x,"")
    return newstr

#Removing Diacritic
def deNoise(text):
    text = str(text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', str(text))
    return text

#Letters Conversion
def transString(string, reverse=1):
    buck2uni = {"'":"ء", "|":"آ", "?":"أ", "&":"ؤ", "<":"إ", "}":"ئ", "A":"ا", "b":"ب",
                "p":"ة", "t":"ت", "v":"ث", "g":"ج", "H":"ح", "x":"خ", "d":"د", "*":"ذ",
                "r":"ر", "z":"ز", "s":"س", "$":"ش", "S":"ص", "D":"ض", "T":"ط", "Z":"ظ",
                "E":"ع", "G":"غ", "_":"ـ", "f":"ف", "q":"ق", "k":"ك", "l":"ل", "m":"م", "n":"ن",
                "h":"ه", "w":"و", "Y":"ى", "y":"ي", "F":"ً", "N":"ٌ", "K":"ٍ", "~":"ّ", "o":"ْ", "u":"ُ",
                "a":"َا", "i":"ِ",
                "e":"ا",
               "2":"ق",
               "3":"ع",
                "4":"ش",
               "5":"خ",
               "7":"ح",
               "c":""}    
    for k, v in buck2uni.items():
        if not reverse:
            string = string.replace(v, k)
        else:
            string = string.replace(k, v)
    return string


# Using only apply function

In [4]:
#Read Data and insert into DF
whatsapp_chat = pre(read_data(directory))
whatsapp_chat_20 = whatsapp_chat[:20]
#whatsapp_chat_20

In [5]:
start_time = time.time()
whatsapp_chat['Chat'] = whatsapp_chat['Chat'].str.lower()
whatsapp_chat = whatsapp_chat.dropna()
whatsapp_chat['Chat'] = whatsapp_chat['Chat'].apply(duos)
whatsapp_chat['Chat'] = whatsapp_chat['Chat'].apply(transString)
whatsapp_chat['Chat'] = whatsapp_chat['Chat'].apply(deNoise)
whatsapp_chat['Chat'] = whatsapp_chat['Chat'].apply(st.stem)
whatsapp_chat['Chat'] = whatsapp_chat['Chat'].apply(anti_vowel)
print(whatsapp_chat['Chat'][:10])
end_time = time.time()
time_elapsed = end_time-start_time
print('The runtime without multiprocess is %s seconds' %(time_elapsed))

0             خمل رتد جرة "برث عل ðÿ“مخ"
1                                   ر ددد
2                    لل مزج جب نزل بدر قل
3      منت ل بتبرد تزخ شش مخنشش جلخ ل مزق
8                        ðÿ˜‚ðÿ˜‚ðÿ˜‚ðÿ˜‚
9                                 نزلن مت
10                ن حتم شنجب خل نتلع عل ش
11                      خ ن شخل نزل عل تل
12                                      خ
13                           ن شت مل نشرد
Name: Chat, dtype: object
The runtime without multiprocess is 98.56512784957886 seconds


# 0.Pandas with Lambda function

In [6]:
#Read Data and insert into DF
whatsapp_chat = pre(read_data(directory))
whatsapp_chat_20 = whatsapp_chat[:20]
#whatsapp_chat_20

In [7]:
def all_in_one(df):
    df['Chat'] = df['Chat'].dropna().apply(lambda x:anti_vowel(st.stem(deNoise(transString(duos(x).lower(), reverse = 1)))))
    return df#.dropna(how = 'any', axis = 0, inplace = True)


In [8]:
start_time = time.time()
Chat = all_in_one(whatsapp_chat)
print(Chat[:10])
end_time = time.time()
time_elapsed = end_time-start_time
print('The runtime without multiprocess is %s seconds' %(time_elapsed))

                  Date                                 Chat
0   12/26/18, 9:57 AM           كمل رتد جرة "برث عل ðÿ“مخ"
1   12/26/18, 9:57 AM                                 ر ددد
2    10/2/20, 4:15 AM                  لل مزج جب نزل بدر قل
3    10/2/20, 4:24 AM    منت ل بتبرد تزخ شش مخنشش جلخ ل مزق
8    10/2/20, 7:21 AM                      ðÿ˜‚ðÿ˜‚ðÿ˜‚ðÿ˜‚
9    10/2/20, 8:56 AM                               نزلن مت
10   10/2/20, 8:56 AM               ن حتم شنجب خل نتلع عل ش
11   10/2/20, 8:57 AM                     خ ن شخل نزل عل تل
12   10/2/20, 8:57 AM                                     خ
13   10/2/20, 9:06 AM                          ن شت مل نشرد
The runtime without multiprocess is 100.25171136856079 seconds


# 1.Multiprocessing 

In [9]:
whatsapp_chat = pre(read_data(directory))
whatsapp_chat_20 = whatsapp_chat[:20]

In [10]:
cores = cpu_count() #Gets number of CPU cores on your machine
partitions = cores #Define number of partitions

def parallelize(df, func):
    df_split = np.array_split(df, partitions)
    pool = Pool(cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df


In [11]:
start_time = time.time()
Chat = parallelize(whatsapp_chat, all_in_one)
end_time = time.time()
print(Chat[:10])
time_elapsed = end_time-start_time
print('The runtime with multiprocess is %s seconds' %(time_elapsed))

                  Date                                 Chat
0   12/26/18, 9:57 AM           كمل رتد جرة "برث عل ðÿ“مخ"
1   12/26/18, 9:57 AM                                 ر ددد
2    10/2/20, 4:15 AM                  لل مزج جب نزل بدر قل
3    10/2/20, 4:24 AM    منت ل بتبرد تزخ شش مخنشش جلخ ل مزق
8    10/2/20, 7:21 AM                      ðÿ˜‚ðÿ˜‚ðÿ˜‚ðÿ˜‚
9    10/2/20, 8:56 AM                               نزلن مت
10   10/2/20, 8:56 AM               ن حتم شنجب خل نتلع عل ش
11   10/2/20, 8:57 AM                     خ ن شخل نزل عل تل
12   10/2/20, 8:57 AM                                     خ
13   10/2/20, 9:06 AM                          ن شت مل نشرد
The runtime with multiprocess is 96.9687294960022 seconds


# 2.Pandarallel

In [12]:
whatsapp_chat = pre(read_data(directory))
whatsapp_chat_20 = whatsapp_chat[:20]

In [13]:
start_time = time.time()
from pandarallel import pandarallel
whatsapp_chat['Chat'] = whatsapp_chat['Chat'].str.lower()
whatsapp_chat = whatsapp_chat.dropna()
pandarallel.initialize(nb_workers=4)
whatsapp_chat['Chat'] = whatsapp_chat['Chat'].parallel_apply(duos)
pandarallel.initialize(nb_workers=4)
whatsapp_chat['Chat'] = whatsapp_chat['Chat'].parallel_apply(transString)
pandarallel.initialize(nb_workers=4)
whatsapp_chat['Chat'] = whatsapp_chat['Chat'].parallel_apply(deNoise)
pandarallel.initialize(nb_workers=4)
whatsapp_chat['Chat'] = whatsapp_chat['Chat'].parallel_apply(st.stem)
pandarallel.initialize(nb_workers=4)
whatsapp_chat['Chat'] = whatsapp_chat['Chat'].parallel_apply(anti_vowel)
end_time = time.time()
print(whatsapp_chat[:10])
time_elapsed = end_time-start_time
print('The runtime with multiprocess is %s seconds' %(time_elapsed))

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
                  Date                                 Chat
0   12/26/18, 9:57 AM           خمل رتد جرة "برث عل ðÿ“مخ"
1   12/26/18, 9:57 AM                                 ر ددد
2    10/2/20, 4:15 AM                  لل مزج جب نزل بدر قل
3    10/2/20, 4:24 AM    منت ل بتبرد تزخ شش مخنشش 

# 3.Multiprocess and Pool

In [14]:
whatsapp_chat = pre(read_data(directory))
whatsapp_chat_20 = whatsapp_chat[:20]

In [15]:
num_partitions = 10 #number of partitions to split dataframe
num_cores = 6 #number of cores on your machine

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [16]:
start_time = time.time()
paraller = parallelize_dataframe(whatsapp_chat, all_in_one)
end_time = time.time()
print(paraller[:10])
time_elapsed = end_time-start_time
print('The runtime with multiprocess and pool is %s seconds' %(time_elapsed))

                  Date                                 Chat
0   12/26/18, 9:57 AM           كمل رتد جرة "برث عل ðÿ“مخ"
1   12/26/18, 9:57 AM                                 ر ددد
2    10/2/20, 4:15 AM                  لل مزج جب نزل بدر قل
3    10/2/20, 4:24 AM    منت ل بتبرد تزخ شش مخنشش جلخ ل مزق
8    10/2/20, 7:21 AM                      ðÿ˜‚ðÿ˜‚ðÿ˜‚ðÿ˜‚
9    10/2/20, 8:56 AM                               نزلن مت
10   10/2/20, 8:56 AM               ن حتم شنجب خل نتلع عل ش
11   10/2/20, 8:57 AM                     خ ن شخل نزل عل تل
12   10/2/20, 8:57 AM                                     خ
13   10/2/20, 9:06 AM                          ن شت مل نشرد
The runtime with multiprocess and pool is 96.77804064750671 seconds


# 4.Swifter

In [17]:
whatsapp_chat = pre(read_data(directory))
whatsapp_chat_20 = whatsapp_chat[:20]

In [18]:
start_time = time.time()
df = whatsapp_chat.dropna()
df['Chat'] = df['Chat'].swifter.apply(lambda x:anti_vowel(st.stem(deNoise(transString(duos(x).lower(), reverse = 1)))))
end_time = time.time()
print(df[:10])
time_elapsed = end_time-start_time
print('The runtime with multiprocess and pool is %s seconds' %(time_elapsed))


Pandas Apply:   0%|          | 0/2903111 [00:00<?, ?it/s]

                  Date                                 Chat
0   12/26/18, 9:57 AM           كمل رتد جرة "برث عل ðÿ“مخ"
1   12/26/18, 9:57 AM                                 ر ددد
2    10/2/20, 4:15 AM                  لل مزج جب نزل بدر قل
3    10/2/20, 4:24 AM    منت ل بتبرد تزخ شش مخنشش جلخ ل مزق
8    10/2/20, 7:21 AM                      ðÿ˜‚ðÿ˜‚ðÿ˜‚ðÿ˜‚
9    10/2/20, 8:56 AM                               نزلن مت
10   10/2/20, 8:56 AM               ن حتم شنجب خل نتلع عل ش
11   10/2/20, 8:57 AM                     خ ن شخل نزل عل تل
12   10/2/20, 8:57 AM                                     خ
13   10/2/20, 9:06 AM                          ن شت مل نشرد
The runtime with multiprocess and pool is 112.75166964530945 seconds


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
